# Hyperopt and MLFlow implementation

## 1. Simple MLFlow implementation

In [4]:
import pandas as pd
import pickle
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge

from sklearn.metrics import root_mean_squared_error

In [1]:
import mlflow

mlflow.set_tracking_uri("http://127.0.0.1:5001")
mlflow.set_experiment("green-taxi-duration")

<Experiment: artifact_location='/Users/gabi/codes/ml_ops/04_deployment/web_service_mlflow/mlruns/908903344791014962', creation_time=1747301871896, experiment_id='908903344791014962', last_update_time=1747301871896, lifecycle_stage='active', name='green-taxi-duration', tags={}>

In [2]:
# read the data and treat it
# change compression if needed

def read_dataframe(filename, compression='gzip'):
    df = pd.read_csv(filename, compression=compression)

    df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
    df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)

    return df

In [5]:
df_train = read_dataframe('../../02_experiment_tracking/mlflow-examples/data/green_tripdata_2021-01.csv.gz')
df_val = read_dataframe('../../02_experiment_tracking/mlflow-examples/data/green_tripdata_2021-02.csv.gz')

/var/folders/55/x1yqz6851xj6j2f0smdhs4jh0000gn/T/ipykernel_9095/697616853.py:5: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(filename, compression=compression)


In [6]:
len(df_train), len(df_val)

(73908, 61921)

In [7]:
# We will use feature engineering to create a new feature called PU_DO 
df_train['PU_DO'] = df_train['PULocationID'] + '_' + df_train['DOLocationID']
df_val['PU_DO'] = df_val['PULocationID'] + '_' + df_val['DOLocationID']

In [8]:
# set the features we will use
categorical = ['PU_DO'] 
numerical = ['trip_distance']

# We will use the DictVectorizer to convert categorical variables into a format that can be used by the model
dv = DictVectorizer()

train_dicts = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

val_dicts = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

In [9]:
# set the target variable

target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

In [10]:
# We will use the LinearRegression model to train the data
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_val)

root_mean_squared_error(y_val, y_pred)

7.758715210533294

In [15]:
# save the model
with open('models/lin_reg.bin', 'wb') as f_out:
    pickle.dump((dv, lr), f_out)

with open('models/dict_vectorizer.bin', 'wb') as f_out:
    pickle.dump(dv, f_out)

In [ ]:
# save the model with mlflow
with mlflow.start_run():

    mlflow.set_tag("developer", "gabi")

    mlflow.log_param("train-data-path", "./data/green_tripdata_2021-01.csv")
    mlflow.log_param("valid-data-path", "./data/green_tripdata_2021-02.csv")

    alpha = 0.1
    mlflow.log_param("alpha", alpha)
    lr = Lasso(alpha)
    lr.fit(X_train, y_train)

    y_pred = lr.predict(X_val)
    rmse = root_mean_squared_error(y_val, y_pred)
    mlflow.log_metric("rmse", rmse)

    mlflow.log_artifact(local_path="models/dict_vectorizer.bin", artifact_path="models_pickle")
    mlflow.log_artifact(local_path="models/lin_reg.bin", artifact_path="models_pickle")


🏃 View run powerful-grub-548 at: http://127.0.0.1:5001/#/experiments/908903344791014962/runs/9d0c0a7479094f4fbb5f789646d6aa06
🧪 View experiment at: http://127.0.0.1:5001/#/experiments/908903344791014962
